In [1]:
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool
from bs4 import BeautifulSoup
import re
import os
import sys
import gzip
from multiprocessing.dummy import Pool, Queue
import json
import codecs
from tqdm import tqdm

In [7]:
def parse(file):
    info = {}
    info['url'] = file.split('\n')[0]
    url = info['url']
    try:
        soup = BeautifulSoup(file, 'html.parser')
        title = soup.find('title')
        if title:
            info['title'] = re.sub(r'[^\w\s]', '', title.text.lower())
        try:
            headings1 = soup.find_all('h1')
            if len(headings1) > 5:
                l = 5
            else:
                l = len(headings1)
            for i in range(l):
                info['h1_' + str(i)] = re.sub(r'[^\w\s]','', headings1[i].text.lower())
        except:
            message = "No h1 in" + url 
            print(message, file=sys.stderr)
        try:
            headings2 = soup.find_all('h2')
            if len(headings2) > 5:
                l = 5
            else:
                l = len(headings2)
            for i in range(l):
                info['h2_' + str(i)] = re.sub(r'[^\w\s]','', headings2[i].text.lower())
        except:
            message = "No h2 in" + url 
            print(message, file=sys.stderr)
        try:
            headings3 = soup.find_all('h3')
            if len(headings3) > 5:
                l = 5
            else:
                l = len(headings3)
            for i in range(l):
                info['h3_' + str(i)] = re.sub(r'[^\w\s]','', headings3[i].text.lower())
        except:
            message = "No h3 in" + url 
            print(message, file=sys.stderr)
        try:
            p = soup.find_all('p')
            if len(p) > 5:
                l = 5
            else:
                l = len(p)
            for i in range(l):
                info['p_' + str(i)] = re.sub(r'[^\w\s]','', p[i].text.lower())
        except:
            message = "No p in" + url 
            print(message, file=sys.stderr)
    except:
        message = "Parsing error: " + url 
        print(message, file=sys.stderr)
    return info

In [3]:
queue = Queue()
dirname = 'content'
for i in range(1, 28027):
    file = str(i) + ".dat"
    with open(os.path.join(dirname, file), mode='r', encoding='utf-8') as fin:
        queue.put(fin.read())

In [4]:
queue.qsize()

28026

In [8]:
def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.json.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            record = parse(queue.get())
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)
            
            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)

In [9]:
with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 28018/28018 [1:00:53<00:00,  7.67it/s]


# Объединение файлов

In [10]:
import pandas as pd
from itertools import chain
from contextlib import ExitStack
from typing import Generator, Dict, Any

In [11]:
def convert(s):
    try:
        d = json.loads(s)
        return d
    except:
        message = "Some parsing error"
        print(message, file=sys.stderr)

In [12]:
def reader(file):
    with gzip.open(file, mode='r') as f_json:
        f_json = codecs.getreader('utf8')(f_json)
        data = f_json.read().split('\n')
        return [convert(data[i]) for i in range(len(data)-1)] # data[-1] = None.. it's strange 

In [15]:
def records_reader(dirname: str):
    first = reader(dirname + '/part_00000.json.gz')
    second = reader(dirname + '/part_00001.json.gz')
    third = reader(dirname + '/part_00002.json.gz')
    forth = reader(dirname + '/part_00003.json.gz')
    fivth = reader(dirname + '/part_00004.json.gz')
    sixth = reader(dirname + '/part_00005.json.gz')
    seventh = reader(dirname + '/part_00006.json.gz')
    eighth = reader(dirname + '/part_00007.json.gz')
    mas = chain(first, second, third, forth, fivth, sixth, seventh, eighth)
    return mas

In [16]:
df = pd.DataFrame(records_reader('data'))
df.to_csv('hw_3.csv', index=False)

In [17]:
df.head()

,h1_0,h1_1,h1_2,h1_3,h1_4,h2_0,h2_1,h2_2,h2_3,h2_4,...,h3_2,h3_3,h3_4,p_0,p_1,p_2,p_3,p_4,title,url
0,сколько калорий в дыне,NaN,NaN,NaN,NaN,\n\n\njquerydocumentreadyfunction var retina...,состав полезные свойства и противопоказания к ...,калорийность дыни,лечебные свойства юкки,NaN,...,лечебные свойства эвкалипта,NaN,NaN,в числе самых любимых лакомств особенно в летн...,прежде чем узнать сколько калорий в 1 кг дыни ...,в состав дыни входит большое количество микроэ...,несмотря на то что дыня является полезной и вк...,если вас интересует сколько калорий в дыне 100...,сколько калорий в дыне торпеда колхозница скол...,chastnosti.com/skolko-kalorij-v-dyne.html
1,NaN,NaN,NaN,NaN,NaN,основы менеджмента часть третья функции управл...,содержание,стратегическое планирование,планирование реализации стратегии,организация взаимодействия и полномочия,...,облако тегов,наши друзья,о сайте,\nглавная\n,,проект vernikovru это библиотека содержащая в...,помимо доступа к материалам на сайте vernikovr...,\nавтор мескон мх альберт м хедоури ф\n04 авгу...,vernikovru всё о менеджменте и it основы мен...,vernikov.ru/krisis/item/173-osnovy-menegmenta-...
2,тимололмэз,NaN,NaN,NaN,NaN,детектор взаимодействий,поиск мед препарата,профиль пользователя,аналоги по акт веществу,взаимодействие,...,индометацин,толбутамид,миглитол,добавить в список тимололмэз,добавляйте препараты в список чтобы проверить ...,введите название или часть названия одного или...,введите название мед препарата или его часть м...,азарга,тимололмэз,farmugroza.ru/medication/9343-timololmez.html
3,отпуск по уходу за ребенком до 3 лет в 2017 году,NaN,NaN,NaN,NaN,еще раз о декрете,продлеваем отпуск,для тех кто работает по совместительству,размер пособий по уходу за ребенком в 2017 году,в каких случаях могут отказать в выплате пособ...,...,NaN,NaN,NaN,вы уже состоявшаяся мама недавно родившая долг...,дело в том что декретный отпуск делится на нес...,такая градация зависит от длительности и разме...,изначально выходя в декрет россиянка получает ...,рассмотрим некоторые нюансы,отпуск по уходу за ребенком до 3 лет в 2017 го...,rusposobie.ru/po-uxodu-za-rebenkom/otpusk-po-u...
4,\nустав,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,добро пожаловать на,сайт моу кумаркинская начальная школа,детский сад,,устав,устав,gov.cap.ru/home/69/school_kumarkino/p3aa1.html


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28018 entries, 0 to 28017
Data columns (total 22 columns):
h1_0     22894 non-null object
h1_1     4647 non-null object
h1_2     2058 non-null object
h1_3     1331 non-null object
h1_4     1126 non-null object
h2_0     15598 non-null object
h2_1     10530 non-null object
h2_2     8410 non-null object
h2_3     6714 non-null object
h2_4     5387 non-null object
h3_0     14247 non-null object
h3_1     11075 non-null object
h3_2     9360 non-null object
h3_3     8076 non-null object
h3_4     6936 non-null object
p_0      23668 non-null object
p_1      21793 non-null object
p_2      20846 non-null object
p_3      20213 non-null object
p_4      19665 non-null object
title    28015 non-null object
url      28018 non-null object
dtypes: object(22)
memory usage: 4.7+ MB
